In [1]:
#General
import numpy as np
import pandas as pd

#Data Retrieval and Visualization
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

#Statistical Package
from scipy import stats 
import statsmodels.api as sm
import math
from scipy.stats import linregress
from scipy.stats import f

/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
from pandas_datareader.famafrench import get_available_datasets
#spy_etf=web.DataReader('SPY','yahoo','2010-01-01','2020-01-01' ) 
beg1='1963-07-01'
end1='2014-12-01' #'2014-12-01'

In [3]:
def fm_summary(p): #Summary Statistics Function
    s = p.describe().T
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    return s[['mean','std','tstat']].T
def update_withrf(dataport):
    for x in range(len(dataport.columns)):
        dataport.iloc[:,x]=dataport.iloc[:,x]-dataport['RF']
    dataport.drop('RF',axis=1,inplace=True)
        

In [4]:
len(get_available_datasets())
#print(ds['DESCR'])
datasets = get_available_datasets()
datasets

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [5]:
#Extracting data
betaPort=web.DataReader('25_Portfolios_ME_BETA_5x5','famafrench',start=beg1,end=end1)
niPort=web.DataReader('25_Portfolios_ME_NI_5x5','famafrench',start=beg1,end=end1)
varPort=web.DataReader('25_Portfolios_ME_VAR_5x5','famafrench',start=beg1,end=end1)
rVarPort=web.DataReader('25_Portfolios_ME_RESVAR_5x5','famafrench',start=beg1,end=end1)
acPort=web.DataReader('25_Portfolios_ME_AC_5x5','famafrench',start=beg1,end=end1)
momPort=web.DataReader('25_Portfolios_ME_Prior_12_2','famafrench',start=beg1,end=end1)
FF5=web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start=beg1,end=end1)
MOM=web.DataReader('F-F_Momentum_Factor','famafrench',start=beg1,end=end1)


In [6]:
betaPort

{0:          SMALL LoBETA  ME1 BETA2  ME1 BETA3  ME1 BETA4  SMALL HiBETA  \
 Date                                                                   
 1963-07       -0.6156    -0.5950     0.1464    -0.4538       -1.0076   
 1963-08        1.2546     4.0686     3.6955     2.0609        3.2983   
 1963-09       -0.8082    -2.0279     0.0034    -1.5411       -3.2312   
 1963-10       -0.2628     1.5439     0.2554     4.2260        2.5346   
 1963-11        0.6858    -2.6122    -0.5701    -1.4607       -1.6136   
 ...               ...        ...        ...        ...           ...   
 2014-08        3.6847     4.0526     4.4137     2.9171        3.6693   
 2014-09       -3.6194    -5.0385    -6.1376    -6.8540       -9.8558   
 2014-10        5.9265     7.5963     6.8120     5.1069        1.9413   
 2014-11       -0.9760    -1.5549    -0.6512    -0.8763       -1.0257   
 2014-12        5.8356     4.0965     5.3248     3.0467        3.3079   
 
          ME2 BETA1  ME2 BETA2  ME2 BETA3  ME2

In [7]:
FF5[0]
FF6=pd.concat([FF5[0],MOM[0]],axis=1,sort=False)

In [8]:
FF6.columns

Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   '], dtype='object')

## Table 1

In [9]:
Table1=fm_summary(FF6).copy()
Table1.rename(str.upper,axis='columns',inplace=True)
Table1.drop('RF',axis=1)

,MKT-RF,SMB,HML,RMW,CMA,MOM
mean,0.507573,0.264531,0.377816,0.267136,0.322136,0.687573
std,4.456559,3.048489,2.818926,2.232324,2.019874,4.220075
tstat,2.831346,2.157177,3.331888,2.974878,3.964689,4.050352


## Table 2 

In [10]:
#Excess return of each anomaly portfolio
betaPortXs = betaPort[0].sub(FF5[0]['RF'],axis=0)
niPortXs = niPort[0].sub(FF5[0]['RF'],axis=0)
varPortXs = varPort[0].sub(FF5[0]['RF'],axis=0)
rVarPortXs = rVarPort[0].sub(FF5[0]['RF'],axis=0)
acPortXs = acPort[0].sub(FF5[0]['RF'],axis=0)
momPortXs = momPort[0].sub(FF5[0]['RF'],axis=0)


In [11]:
def GRS(R,F): #GRS Statistics
    R=R.astype('float64')
    F=F.astype('float64')
    alphas = []
    alpha2=[]
    alphasquared=[]
    errors = []
    betas=[]
    returns=[]
    risquared=[]
    rsquared=[]
    bsealphas=[]
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alphas.append(fit.params["const"])
        alpha2.append(abs(fit.params["const"]))
        alphasquared.append(abs(fit.params["const"])**2)
        risquared.append((R[:][p].mean()-F['Mkt-RF'].mean())**2)
        betas.append(fit.params[1:])
        errors.append(fit.resid)
        returns.append(R[:][p].mean()-F['Mkt-RF'].mean())
        
        rsquared.append(fit.rsquared_adj)
        bsealphas.append(fit.bse["const"]**2)
   #For remaining statistics to compute mean. Convert list to arrasy
    alpha2=pd.np.array(alpha2)
    alphasquared=pd.np.array(alphasquared)
    returns=pd.np.array(returns)
    rsquared=pd.np.array(rsquared)
    bsealphas=pd.np.array(bsealphas)
    risquared=pd.np.array(risquared)
    #For GRS
    alphas = pd.np.array(alphas)
    errors = pd.np.array(errors).transpose()
    Ri=returns.mean()
    K = F.shape[1]
    T, N = errors.shape

    Ve = errors.transpose().dot(errors) / (T - K - 1)#unbiased estimate of the covariance matrix of residuals

    r = F.mean().values #step 5Factor mean vector

    f = F.values

    Vf = (f-r).transpose().dot(f-r) / (T - 1) #Step 7 unbiased estimate of the covariance matrix of the factors

    Sh2_a = alphas.dot(pd.np.linalg.pinv(Ve)).dot(alphas)
    Sh2_f = r.dot(pd.np.linalg.pinv(Vf)).dot(r)
    Sh2_a, Sh2_f
    ans=(T/N)*((T-N-K)/(T-K-1))*(Sh2_a/(1+Sh2_f))
    ans=ans.astype(float)
    #print("P-value of the GRS statistics: ",(1-f.cdf(ans,N,T-N-K)))
    #print("Average Absolute value of intercept A|ai|: ",alpha2.mean())
    #print("Average Absolute value of intercept/return A|ai|/A|ri| : ",alpha2.mean()/Ri)
    #print("Average Absolute value of intercept/return squared: ",(alpha2.mean()**2/Ri**2))
    #print("Average Absolute value of the As^2(ai)/A(ai)^2: ",(bsealphas.mean()))
    #print("Average value of the regression R^2 corrected for degress of freedom: ",rsquared.mean())
    return (ans,N,T-N-K,alpha2.mean(),alpha2.mean()/Ri,(alphasquared.mean()/risquared.mean()),(bsealphas.mean()/alphasquared.mean()),rsquared.mean())

In [12]:
#Factors
F1=FF5[0][['Mkt-RF']] #For portfolio beta only
F2=FF5[0][['Mkt-RF','SMB','HML']]
F3=FF5[0][['Mkt-RF','SMB','HML','RMW']]
F4=FF5[0][['Mkt-RF','SMB','HML','CMA']]
F5=FF5[0][['Mkt-RF','SMB','RMW','CMA']]
F6=FF5[0][['Mkt-RF','SMB','HML','RMW','CMA']]
#For Momentum Anomaly only
F71=FF6[['Mkt-RF','SMB','HML','Mom   ']]
F72=FF6[['Mkt-RF','SMB','HML','RMW','Mom   ']]
F73=FF6[['Mkt-RF','SMB','HML','CMA','Mom   ']]
F74=FF6[['Mkt-RF','SMB','RMW','CMA','Mom   ']]
F75=FF6[['Mkt-RF','SMB','HML','RMW','CMA','Mom   ']]

In [13]:
result=GRS(betaPortXs,F1)
#1-f.cdf(GRS(betaPortXs,F1)[0],GRS(betaPortXs,F1)[1],GRS(betaPortXs,F1)[2])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

In [14]:
#Index name for GRS-Statistics
def indexname(f):
    name=""
    for data in list(f.columns):
        name=name+str(data)+" "
    return name
#Initialize
factorList=[F1,F2,F3,F4,F5,F6]
momfactorlist=[F2,F3,F4,F5,F6,F71,F72,F73,F74,F75]
portfolioList=[betaPortXs,niPortXs,varPortXs,rVarPortXs,acPortXs]
portnameList=["35 Size-NI Portffolio","25 Size-Var Portfolio","25 Size-RVar portfolios","25 Size-AC portfolios"] 
column_names=["Model Factors" , "GRS","p(GRS)" ,"A|ai|" ,"A|ai|/A|ri|","Aai^2/Ari^2","As^2(ai)/Aasi^2","A(R^2)"]
df=pd.DataFrame(columns=column_names)
counter=0

In [15]:
df.loc[0]=['25 Size Beta Portfolio',"","","","","","",""]
for fa in factorList:
    ans=GRS(betaPortXs,fa)
    counter+=1
    df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  

for port,name in list(zip(portfolioList[1:],portnameList)):
    counter+=1
    df.loc[counter]=[name,"","","","","","",""]
    for fa in factorList[1:]:
            ans=GRS(port,fa)
            counter+=1
            df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  
counter+=1       
df.loc[counter]=['25 Size Prior2-12 portfolios',"","","","","","",""]
for fa in momfactorlist:
    ans=GRS(momPortXs,fa)
    counter+=1
    df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

In [16]:
df.set_index('Model Factors',drop=True,inplace=True)
#print (df.to_string(index=False))

In [17]:
df

,GRS,p(GRS),A|ai|,A|ai|/A|ri|,Aai^2/Ari^2,As^2(ai)/Aasi^2,A(R^2)
Model Factors,,,,,,,
25 Size Beta Portfolio,,,,,,,
Mkt-RF,2.24577,0.0005437,0.23209,1.03447,0.993389,0.177559,0.750907
Mkt-RF SMB HML,1.58417,0.0362324,0.106704,0.475602,0.200599,0.378613,0.885964
Mkt-RF SMB HML RMW,1.79938,0.0103335,0.0805971,0.359239,0.126284,0.583405,0.892283
Mkt-RF SMB HML CMA,1.47396,0.0652669,0.092491,0.412252,0.164151,0.468494,0.886961
Mkt-RF SMB RMW CMA,1.84083,0.00800181,0.0730447,0.325576,0.1031,0.76288,0.887233
Mkt-RF SMB HML RMW CMA,1.85233,0.00745084,0.0731492,0.326041,0.103211,0.726747,0.89411
35 Size-NI Portffolio,,,,,,,
Mkt-RF SMB HML,4.26184,6.58362e-14,0.138375,0.718543,0.40317,0.180266,0.870902


### 2.3 The five factor model vs FF three factor model

In [18]:
RMWexposure=GRS(FF5[0][['RMW']],F2)
CMAexposure=GRS(FF5[0][['CMA']],F2)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

In [19]:
RMWexposure
CMAexposure

(12.157033800962251,
 1,
 614,
 0.19807551798154666,
 -1.0681559346648823,
 1.1409571007598087,
 0.08226278493015335,
 0.5327048198723323)

### 2.4 An equivalent five factor model

In [20]:
#Equivalent five factore model
HMLexposure=GRS(FF5[0][['HML']],F5)
HMLexposure

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

(0.00034949675711549473,
 1,
 613,
 0.0015780009655745886,
 -0.012161174669224279,
 0.00014789416933538228,
 2861.7162799925095,
 0.5007869389352935)

In [21]:
model = sm.OLS(FF5[0][['HML']], sm.add_constant(F5))

In [22]:
model.fit().summary() #Constant is around 0.0016, does not really add value or means that nothing is lost in explanation of average returns 
#if we drop HML from the model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    HML   R-squared:                       0.504
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     155.7
Date:                Sun, 25 Oct 2020   Prob (F-statistic):           7.09e-92
Time:                        23:55:03   Log-Likelihood:                -1300.2
No. Observations:                 618   AIC:                             2610.
Df Residuals:                     613   BIC:                             2633.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0016      0.084     -0.019      0.985      -0.167       0.164
Mkt-RF         0.0170      0.021      0.824      0.410      -0.023       0.057
SMB            0.0268      0.029      0.929      0.353      -0.030       0.083
RMW            0.1509      0.039      3.856      0.000       0.074       0.228
CMA            1.0039      0.044     23.013      0.000       0.918       1.090
==============================================================================
Omnibus:                       42.373   Durbin-Watson:                   1.725
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              145.707
Skew:                          -0.205   Prob(JB):                     2.29e-32
Kurtosis:                       5.343   Cond. No.                         5.08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
##Calculate HMLO
HMLO=model.fit().params[0]+model.fit().resid
HMLO

Date
1963-07    0.244033
1963-08    1.946285
1963-09    0.093266
1963-10    1.725205
1963-11   -0.413765
             ...   
2014-08    0.124756
2014-09   -0.842320
2014-10   -1.722604
2014-11   -3.267601
2014-12    1.396146
Freq: M, Length: 618, dtype: float64

## 3. Market Beta

In [24]:
dt3=betaPortXs.mean()
dt3
dt3sd=betaPortXs.std()
dt3sd

SMALL LoBETA    4.404342
ME1 BETA2       5.092372
ME1 BETA3       5.946717
ME1 BETA4       6.514479
SMALL HiBETA    8.230431
ME2 BETA1       4.303275
ME2 BETA2       4.776218
ME2 BETA3       5.472472
ME2 BETA4       6.204383
ME2 BETA5       7.914249
ME3 BETA1       3.859098
ME3 BETA2       4.643507
ME3 BETA3       5.228561
ME3 BETA4       5.985849
ME3 BETA5       7.661497
ME4 BETA1       3.881337
ME4 BETA2       4.623011
ME4 BETA3       5.168256
ME4 BETA4       5.838906
ME4 BETA5       7.589291
BIG LoBETA      3.627947
ME5 BETA2       4.254481
ME5 BETA3       4.874660
ME5 BETA4       5.703102
BIG HiBETA      7.144848
dtype: float64

In [25]:
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a 

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,Mean,,,,,,SD,,
small,0.729199,0.89061,0.928194,0.971773,0.7968,,4.40434,5.09237,5.94672,6.51448,8.23043
2,0.70595,0.867878,0.950831,0.884945,0.717472,,4.30328,4.77622,5.47247,6.20438,7.91425
3,0.689102,0.862609,0.837951,0.805145,0.752985,,3.8591,4.64351,5.22856,5.98585,7.6615
4,0.669165,0.761869,0.734495,0.586254,0.752701,,3.88134,4.62301,5.16826,5.83891,7.58929
Big,0.488234,0.516946,0.495187,0.497284,0.404631,,3.62795,4.25448,4.87466,5.7031,7.14485


In [26]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=betaPort[6].mean()
dt3b=betaPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,B/M,,,,,,OP,,
small,0.978333,1.02833,1.03239,1.03212,0.935291,,0.166656,0.157609,0.169663,0.79097,0.0451992
2,0.85808,0.85243,0.826256,0.803328,0.739835,,0.266445,0.260208,0.376776,0.341499,0.241039
3,0.81999,0.773257,0.72836,0.695405,0.661341,,0.272471,0.308223,0.338014,0.343829,0.327291
4,0.756989,0.686216,0.668542,0.666935,0.6423,,0.314529,0.382237,0.321304,0.342551,0.323477
Big,0.611213,0.477666,0.543125,0.532282,0.616422,,0.403892,0.467218,0.3724,0.366538,0.331539


In [27]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=betaPort[10].mean()
dt3b=betaPort[8].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','Prior-Beta',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,Inv,,,,,,Prior-Beta,,
small,0.0975382,0.0872642,0.0964901,0.0950502,0.123247,,0.321018,0.836558,1.15124,1.5117,2.47944
2,0.110291,0.107712,0.122171,0.135044,0.177786,,0.380648,0.83935,1.15423,1.50746,2.39704
3,0.110965,0.115324,0.124481,0.142184,0.194583,,0.382849,0.838484,1.15055,1.49794,2.29605
4,0.0990663,0.110462,0.118131,0.129168,0.186805,,0.397009,0.840444,1.14688,1.49709,2.23278
Big,0.0906003,0.110098,0.133069,0.142191,0.181013,,0.422509,0.828333,1.14409,1.48018,2.0965


In [28]:
##Continue to the deeper regression on each port
def regCAPM(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
    return (alpha,beta,tstata,tstatb)
            
def regFivefactor(R,F):
    alpha=[]
    beta=[]
    s=[]
    h=[]
    r=[]
    c=[]
    tstata=[]
    tstatb=[]
    tstats=[]
    tstath=[]
    tstatr=[]
    tstatc=[]
       
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        s.append(fit.params[2])
        h.append(fit.params[3])
        r.append(fit.params[4])
        c.append(fit.params[5])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        tstats.append(fit.tvalues[2])
        tstath.append(fit.tvalues[3])
        tstatr.append(fit.tvalues[4])
        tstatc.append(fit.tvalues[5])
    return (alpha,beta,tstata,tstatb,s,tstats,h,tstath,r,tstatr,c,tstatc)

In [29]:

result3a=regCAPM(betaPortXs,F1)


In [30]:
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24])]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4a

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,a,,,,,,t(a),,
small,0.347428,0.434371,0.398515,0.359296,0.046138,,3.00121,3.40958,2.65382,2.41163,0.230992
2,0.316107,0.410705,0.41022,0.271152,-0.0689121,,2.99124,3.91597,3.71733,2.17677,-0.439677
3,0.326199,0.389035,0.303112,0.195004,-0.0264082,,3.69774,4.64483,3.25376,1.80213,-0.189175
4,0.303572,0.278319,0.186251,-0.0303556,-0.025644,,3.43362,3.75464,2.44322,-0.342576,-0.191747
Big,0.150421,0.0673103,-0.0304781,-0.112867,-0.314429,,1.77696,1.0475,-0.47955,-1.42415,-2.31985
,,,b,,,,,,t(b),,
small,0.75215,0.898864,1.04355,1.20668,1.47893,,29.1198,31.6217,31.1452,36.2995,33.1845
2,0.768053,0.900703,1.06509,1.20927,1.5493,,32.5732,38.4895,43.2564,43.5086,44.3023
3,0.714977,0.933017,1.05372,1.20207,1.53553,,36.3243,49.9254,50.6943,49.7881,49.2985


In [31]:
## Five Factor regression with HML->HMLO
test=F2.copy()
test=test.drop(['HML'],axis=1)
test['HMLO']=HMLO
test['RMW']=F5['RMW']
test['CMA']=F5['CMA']

In [32]:
result3a=regFivefactor(betaPortXs,test) #For five factor regression

In [33]:
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,a,,,,,,t(a),,
small,0.0807789,0.0738599,0.0611103,0.0781181,0.00143933,,1.22577,1.1465,0.927991,1.28732,0.0172874
2,0.0292962,0.0334786,0.0602489,-0.0612311,-0.14056,,0.396538,0.608959,1.03365,-1.03302,-1.86162
3,0.0685787,0.0942608,-0.00393585,-0.104428,-0.0233097,,0.90988,1.59809,-0.0636578,-1.4836,-0.259726
4,0.0445253,-0.0250222,-0.0943501,-0.247718,0.0974156,,0.543666,-0.39734,-1.44958,-3.15382,0.880232
Big,-0.0827952,-0.084933,-0.0891004,-0.0929662,-0.0552695,,-1.2577,-1.61037,-1.50589,-1.14504,-0.406202
,,,b,,,,,,t(b),,
small,0.663388,0.809852,0.895711,1.02946,1.14414,,41.2868,51.5588,55.7865,69.579,56.3614
2,0.730034,0.861166,0.996812,1.09768,1.29476,,40.5273,64.2452,70.1408,75.9529,70.3315
3,0.749611,0.922226,1.01915,1.13236,1.31025,,40.7908,64.1265,67.6054,65.9802,59.8775


### 4.NI Net Share Issues

In [38]:
dt3=niPortXs.mean()
dt3
dt3sd=niPortXs.std()
dt3sd.head()

SMALL NegNI     5.667709
SMALL ZeroNI    5.726147
SMALL LoNI      6.362067
ME1 NI2         6.432492
ME1 NI3         6.857032
dtype: float64

In [37]:
# List of tuples
students = [ (' ',' ',' ','Mean',' ',' ',' ',' ',' ',' ',' ','SD',' ',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],dt3[5], dt3[6],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4],dt3sd[5], dt3sd[6]),
             (dt3[7], dt3[8], dt3[9] , dt3[10],dt3[11],dt3[12], dt3[13],' ',dt3sd[7], dt3sd[8], dt3sd[9] , dt3sd[10],dt3sd[11],dt3sd[12], dt3sd[13]),
             (dt3[14], dt3[15], dt3[16],dt3[17],dt3[18],dt3[19], dt3[20],' ',dt3sd[14], dt3sd[15], dt3sd[16] , dt3sd[17],dt3sd[18],dt3sd[19], dt3sd[20]),
             (dt3[21], dt3[22], dt3[23] , dt3[24],dt3[25],dt3[26], dt3[27],' ',dt3sd[21], dt3sd[22], dt3sd[23] , dt3sd[24],dt3sd[25],dt3sd[26], dt3sd[27]),
             (dt3[28], dt3[29], dt3[30] , dt3[31],dt3[32],dt3[33], dt3[34],' ',dt3sd[28], dt3sd[29], dt3sd[30] , dt3sd[31],dt3sd[32],dt3sd[33], dt3sd[34])]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a 

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,Mean,,,,,,,,SD,,,
small,1.04094,0.781797,0.899782,0.950428,1.02083,0.698482,0.242634,,5.66771,5.72615,6.36207,6.43249,6.85703,7.12108,7.73782
2,0.911954,0.802531,0.91439,0.913345,0.850793,0.81269,0.306043,,5.30514,5.63293,5.83911,6.1276,6.32045,6.53043,7.07975
3,0.963668,0.751828,0.833342,0.927842,0.832749,0.72355,0.286675,,5.07919,5.43081,5.28315,5.63099,5.83469,6.0131,6.54849
4,0.941052,0.560273,0.653033,0.687051,0.818819,0.621508,0.363179,,4.91186,4.88524,5.07014,5.33553,5.45465,5.80948,6.1861
Big,0.615858,0.608861,0.490563,0.479228,0.590431,0.431733,0.177375,,4.17591,4.78653,4.24836,4.55064,4.85929,5.41809,5.12104
